In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from google.colab import drive
drive.mount('/content/drive')
train_csv_path = "/content/drive/MyDrive/UIUC-CS/CS412/Final Project/train.csv"
#train_csv_path = '/kaggle/input/bigquery-geotab-intersection-congestion/train.csv'
df = pd.read_csv(train_csv_path)
print(df.head())

Mounted at /content/drive
     RowId  IntersectionId   Latitude  Longitude  \
0  1921357               0  33.791659 -84.430032   
1  1921358               0  33.791659 -84.430032   
2  1921359               0  33.791659 -84.430032   
3  1921360               0  33.791659 -84.430032   
4  1921361               0  33.791659 -84.430032   

                EntryStreetName                ExitStreetName EntryHeading  \
0  Marietta Boulevard Northwest  Marietta Boulevard Northwest           NW   
1  Marietta Boulevard Northwest  Marietta Boulevard Northwest           SE   
2  Marietta Boulevard Northwest  Marietta Boulevard Northwest           NW   
3  Marietta Boulevard Northwest  Marietta Boulevard Northwest           SE   
4  Marietta Boulevard Northwest  Marietta Boulevard Northwest           NW   

  ExitHeading  Hour  Weekend  ...  TimeFromFirstStop_p40  \
0          NW     0        0  ...                    0.0   
1          SE     0        0  ...                    0.0   
2          N

In [3]:
df = pd.read_csv(train_csv_path)
x_train = df[['IntersectionId','EntryHeading','ExitHeading']]
x_train = pd.concat([x_train,pd.get_dummies(df["City"],dummy_na=False, drop_first=False, dtype=int)],axis=1)
print(x_train.head())
heading_map = {'N':1,'NE':2,'E':3,'SE':4,'S':5, 'SW':6, 'W':7, 'NW': 8}
x_train['EntryHeading'] = x_train['EntryHeading'].replace(heading_map)
x_train['ExitHeading'] = x_train['ExitHeading'].replace(heading_map)
x_train['EntryHeading'] = x_train['EntryHeading'].replace(heading_map)
x_train['ExitHeading'] = x_train['ExitHeading'].replace(heading_map)
print(x_train)


   IntersectionId EntryHeading ExitHeading  Atlanta  Boston  Chicago  \
0               0           NW          NW        1       0        0   
1               0           SE          SE        1       0        0   
2               0           NW          NW        1       0        0   
3               0           SE          SE        1       0        0   
4               0           NW          NW        1       0        0   

   Philadelphia  
0             0  
1             0  
2             0  
3             0  
4             0  


/tmp/ipython-input-1820002226.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x_train['EntryHeading'] = x_train['EntryHeading'].replace(heading_map)


        IntersectionId  EntryHeading  ExitHeading  Atlanta  Boston  Chicago  \
0                    0             8            8        1       0        0   
1                    0             4            4        1       0        0   
2                    0             8            8        1       0        0   
3                    0             4            4        1       0        0   
4                    0             8            8        1       0        0   
...                ...           ...          ...      ...     ...      ...   
856382            1990             5            7        0       0        0   
856383            1990             5            5        0       0        0   
856384            1990             5            5        0       0        0   
856385            1990             7            7        0       0        0   
856386            1990             5            5        0       0        0   

        Philadelphia  
0                  0  
1    

/tmp/ipython-input-1820002226.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x_train['ExitHeading'] = x_train['ExitHeading'].replace(heading_map)


In [4]:
# Hour: 0-23 cyclical -> [sin(2pi*hour/24), cos(2pi*hour/24)]
df['hour_sin'] = np.sin(2*np.pi*df['Hour']/24)
df['hour_cos'] = np.cos(2*np.pi*df['Hour']/24)
print(df.head())
print(df['Month'].unique())

df['month_sin'] = np.sin(2*np.pi*df['Month']/12)
df['month_cos'] = np.cos(2*np.pi*df['Month']/12)
print(df.head())

     RowId  IntersectionId   Latitude  Longitude  \
0  1921357               0  33.791659 -84.430032   
1  1921358               0  33.791659 -84.430032   
2  1921359               0  33.791659 -84.430032   
3  1921360               0  33.791659 -84.430032   
4  1921361               0  33.791659 -84.430032   

                EntryStreetName                ExitStreetName EntryHeading  \
0  Marietta Boulevard Northwest  Marietta Boulevard Northwest           NW   
1  Marietta Boulevard Northwest  Marietta Boulevard Northwest           SE   
2  Marietta Boulevard Northwest  Marietta Boulevard Northwest           NW   
3  Marietta Boulevard Northwest  Marietta Boulevard Northwest           SE   
4  Marietta Boulevard Northwest  Marietta Boulevard Northwest           NW   

  ExitHeading  Hour  Weekend  ...  TimeFromFirstStop_p60  \
0          NW     0        0  ...                    0.0   
1          SE     0        0  ...                    0.0   
2          NW     1        0  ...     

In [5]:
x_train['hour_sin'] = df['hour_sin']
x_train['hour_cos'] = df['hour_cos']
x_train['month_sin'] = df['month_sin']
x_train['month_cos'] = df['month_cos']
x_train['Weekend'] = df['Weekend']
x_train['Month'] = df['Month']
x_train['Hour'] = df['Hour']
print(x_train)

        IntersectionId  EntryHeading  ExitHeading  Atlanta  Boston  Chicago  \
0                    0             8            8        1       0        0   
1                    0             4            4        1       0        0   
2                    0             8            8        1       0        0   
3                    0             4            4        1       0        0   
4                    0             8            8        1       0        0   
...                ...           ...          ...      ...     ...      ...   
856382            1990             5            7        0       0        0   
856383            1990             5            5        0       0        0   
856384            1990             5            5        0       0        0   
856385            1990             7            7        0       0        0   
856386            1990             5            5        0       0        0   

        Philadelphia  hour_sin      hour_cos     mo

In [6]:
y_train = pd.DataFrame()
y_train['TotalTimeStopped_p20'] = df['TotalTimeStopped_p20']
y_train['TotalTimeStopped_p50'] = df['TotalTimeStopped_p50']
y_train['TotalTimeStopped_p80'] = df['TotalTimeStopped_p80']
y_train['DistanceToFirstStop_p20'] = df['DistanceToFirstStop_p20']
y_train['DistanceToFirstStop_p50'] = df['DistanceToFirstStop_p50']
y_train['DistanceToFirstStop_p80'] = df['DistanceToFirstStop_p80']
print(y_train.head())

   TotalTimeStopped_p20  TotalTimeStopped_p50  TotalTimeStopped_p80  \
0                   0.0                   0.0                   0.0   
1                   0.0                   0.0                   0.0   
2                   0.0                   0.0                   0.0   
3                   0.0                   0.0                   0.0   
4                   0.0                   0.0                   0.0   

   DistanceToFirstStop_p20  DistanceToFirstStop_p50  DistanceToFirstStop_p80  
0                      0.0                      0.0                      0.0  
1                      0.0                      0.0                      0.0  
2                      0.0                      0.0                      0.0  
3                      0.0                      0.0                      0.0  
4                      0.0                      0.0                      0.0  


In [7]:
#Split data into train and val
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
print(x_train.head())
print("len x_train", len(x_train))
print("len x_val",len(x_val))
print(y_train.head())
print("len y_train", len(y_train))
print("len y_val",len(y_val))

        IntersectionId  EntryHeading  ExitHeading  Atlanta  Boston  Chicago  \
538730             345             7            7        0       0        0   
630597             826             2            2        0       0        0   
817258            1796             6            6        0       0        0   
526197             264             5            5        0       0        0   
256532             667             2            3        0       1        0   

        Philadelphia      hour_sin  hour_cos     month_sin  month_cos  \
538730             1 -7.071068e-01 -0.707107 -8.660254e-01   0.500000   
630597             1  9.659258e-01 -0.258819 -8.660254e-01  -0.500000   
817258             1  7.071068e-01 -0.707107 -5.000000e-01   0.866025   
526197             1  5.000000e-01 -0.866025 -2.449294e-16   1.000000   
256532             0  1.224647e-16 -1.000000 -2.449294e-16   1.000000   

        Weekend  Month  Hour  
538730        0     10    15  
630597        0      8  

## Own Method-LSTM

In [8]:
pip install pydot graphviz

In [12]:
SEQ_LEN    = 12    # number of rows per sequence
BATCH_SIZE = 256   # trade-off between gradient stability and memory
EPOCHS     = 5
LR         = 1e-3
SEED       = 42


import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.utils import plot_model

# fixed randomness
np.random.seed(SEED)
tf.random.set_seed(SEED)

# sort by time (Month, Hour)
# since LSTM need chronologically ordered
required_df_cols = ['Month', 'Hour', 'City', 'IntersectionId']
missing_any = False
for c in required_df_cols:
    if c not in df.columns:
        print("ERROR: Missing column in df:", c)
        missing_any = True

feature_cols = list(x_train.columns)   # your engineered X columns (one-hots, headings, cyclical time, etc.)
target_cols  = list(y_train.columns)   # the 6 regression targets

print("Number of feature columns:", len(feature_cols))
print("Feature columns:", feature_cols)
print("Target columns:", target_cols)

if missing_any:
    print("Something is missing.")



def make_sequences_from_split(X_split, Y_split, seq_len):
    """
    Turn tabular X/Y into LSTM-ready sequences:
      - Group by (CityKey, IntersectionId) using one-hot city CityKey
      - Sort by cyclic time via angles: atan2(sin, cos) for month/hour
      - Create sliding windows of length seq_len; label = last row in window
    """
    tmp = X_split.copy()

    # Rebuild a city key from your one-hot columns
    city_cols = [c for c in ['Atlanta','Boston','Chicago','Philadelphia'] if c in tmp.columns]
    if len(city_cols) == 0:
        raise ValueError("City one-hot columns not found in X. Expected any of: Atlanta/Boston/Chicago/Philadelphia.")
    tmp['CityKey'] = tmp[city_cols].idxmax(axis=1)

    # Reconstruct sortable cyclic angles for month/hour from your sin/cos features
    for need in ['hour_sin','hour_cos','month_sin','month_cos']:
        if need not in tmp.columns:
            raise ValueError(f"Missing column '{need}' in X needed to sort time for sequence building.")

    # atan2 → (-pi, pi]; map to [0, 2pi) so it's strictly sortable
    tmp['hour_angle']  = np.mod(np.arctan2(tmp['hour_sin'],  tmp['hour_cos']) + 2.0*np.pi, 2.0*np.pi)
    tmp['month_angle'] = np.mod(np.arctan2(tmp['month_sin'], tmp['month_cos']) + 2.0*np.pi, 2.0*np.pi)

    # Keep feature order consistent and valid
    valid_feat_cols = [c for c in feature_cols if c in tmp.columns]

    # Sort within (City, IntersectionId) by month then hour
    tmp_sorted = tmp.sort_values(['CityKey', 'IntersectionId', 'month_angle', 'hour_angle'])

    # Align Y to the same order
    Y_sorted = Y_split.loc[tmp_sorted.index]

    X_list, Y_list = [], []

    # Group by (CityKey, IntersectionId) so we never mix different locations in one sequence
    for (_, _), gdf in tmp_sorted.groupby(['CityKey','IntersectionId'], sort=False):
        if len(gdf) < seq_len:
            continue

        Xg = gdf[valid_feat_cols].to_numpy(dtype=np.float32)
        Yg = Y_sorted.loc[gdf.index].to_numpy(dtype=np.float32)

        # Sliding window; label is the window's last timestep
        for i in range(seq_len - 1, len(gdf)):
            start = i - seq_len + 1
            X_list.append(Xg[start:i+1])
            Y_list.append(Yg[i])

    if not X_list:
        print("⚠️ No sequences were created. Try reducing SEQ_LEN (e.g., 6).")
        return (
            np.empty((0, seq_len, len(valid_feat_cols)), np.float32),
            np.empty((0, Y_split.shape[1]), np.float32)
        )

    return np.stack(X_list).astype(np.float32), np.stack(Y_list).astype(np.float32)


# Build sequences for train/val (no scaling applied above)
Xtr_seq, ytr_seq = make_sequences_from_split(x_train, y_train, SEQ_LEN)
Xva_seq, yva_seq = make_sequences_from_split(x_val,   y_val,   SEQ_LEN)

print("X_train seq shape:", Xtr_seq.shape, "y_train shape:", ytr_seq.shape)
print("X_val   seq shape:", Xva_seq.shape, "y_val shape:", yva_seq.shape)

if Xtr_seq.shape[0] == 0 or Xva_seq.shape[0] == 0:
    print("No sequences built. Check SEQ_LEN or key columns in df (City/IntersectionId/Month/Hour).")


# LSTM head
def build_head_model(input_seq_len, input_feat_dim, out_dim=3, lstm_units=64, dense_units=128, dropout=0.1):
    model = keras.Sequential()
    model.add(layers.Input(shape=(input_seq_len, input_feat_dim)))
    model.add(layers.LSTM(lstm_units))
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(dense_units, activation="relu"))
    model.add(layers.Dense(out_dim))  # outputs [p20, p50, p80]

    model.compile(
        optimizer=keras.optimizers.Adam(LR),
        loss="mse",
        metrics=[keras.metrics.RootMeanSquaredError(name="rmse")]
    )
    return model

# Prepare labels for TIME and DISTANCE heads (N,3)
time_cols = ['TotalTimeStopped_p20','TotalTimeStopped_p50','TotalTimeStopped_p80']
dist_cols = ['DistanceToFirstStop_p20','DistanceToFirstStop_p50','DistanceToFirstStop_p80']

time_idx = [target_cols.index(c) for c in time_cols]
dist_idx = [target_cols.index(c) for c in dist_cols]

ytr_time = ytr_seq[:, time_idx]
yva_time = yva_seq[:, time_idx]
ytr_dist = ytr_seq[:, dist_idx]
yva_dist = yva_seq[:, dist_idx]

# Callbacks
callbacks = [
    keras.callbacks.ReduceLROnPlateau(monitor="val_rmse", factor=0.5, patience=3, verbose=1),
    keras.callbacks.EarlyStopping(monitor="val_rmse", patience=7, restore_best_weights=True, verbose=1)
]

# Train TIME model
print("\nTraining TIME model...")
time_model = build_head_model(Xtr_seq.shape[1], Xtr_seq.shape[2], out_dim=3)

# plot the model
plot_model(
    time_model,
    show_shapes=True,
    show_layer_names=True
)

time_model.fit(
    Xtr_seq, ytr_time,
    validation_data=(Xva_seq, yva_time),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)


# Train DISTANCE model
print("\nTraining DISTANCE model...")
dist_model = build_head_model(Xtr_seq.shape[1], Xtr_seq.shape[2], out_dim=3)
dist_model.fit(
    Xtr_seq, ytr_dist,
    validation_data=(Xva_seq, yva_dist),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)

# Validation prediction + per-column RMSE
print("\nPredicting on validation set...")

pred_time = time_model.predict(Xva_seq, batch_size=BATCH_SIZE, verbose=0)  # (N, 3)
pred_dist = dist_model.predict(Xva_seq, batch_size=BATCH_SIZE, verbose=0)  # (N, 3)

def rmse_each_column(y_true, y_pred, names):
    scores = {}
    for i, name in enumerate(names):
        mse  = mean_squared_error(y_true[:, i], y_pred[:, i])
        rmse = float(np.sqrt(mse))
        scores[name] = rmse
    return scores

rmse_time = rmse_each_column(yva_time, pred_time, time_cols)
rmse_dist = rmse_each_column(yva_dist, pred_dist, dist_cols)

print("\n=== RMSE===")
for k, v in rmse_time.items():
    print(k.ljust(30), ":", round(v, 4))
for k, v in rmse_dist.items():
    print(k.ljust(30), ":", round(v, 4))

print("\n=== Overall RMSE===")
# Combine time + distance
y_true_all = np.concatenate([yva_time, yva_dist], axis=1)
y_pred_all = np.concatenate([pred_time, pred_dist], axis=1)

overall_mse = np.mean((y_true_all - y_pred_all)**2)
overall_rmse = float(np.sqrt(overall_mse))

print("Overall RMSE:", overall_rmse)

Number of feature columns: 14
Feature columns: ['IntersectionId', 'EntryHeading', 'ExitHeading', 'Atlanta', 'Boston', 'Chicago', 'Philadelphia', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos', 'Weekend', 'Month', 'Hour']
Target columns: ['TotalTimeStopped_p20', 'TotalTimeStopped_p50', 'TotalTimeStopped_p80', 'DistanceToFirstStop_p20', 'DistanceToFirstStop_p50', 'DistanceToFirstStop_p80']
X_train seq shape: (634740, 12, 14) y_train shape: (634740, 6)
X_val   seq shape: (126757, 12, 14) y_val shape: (126757, 6)

Training TIME model...
Epoch 1/5
2480/2480 ━━━━━━━━━━━━━━━━━━━━ 98s 38ms/step - loss: 380.2728 - rmse: 19.4924 - val_loss: 370.3979 - val_rmse: 19.2457 - learning_rate: 0.0010
Epoch 2/5
2480/2480 ━━━━━━━━━━━━━━━━━━━━ 91s 37ms/step - loss: 352.7939 - rmse: 18.7820 - val_loss: 372.5889 - val_rmse: 19.3026 - learning_rate: 0.0010
Epoch 3/5
2480/2480 ━━━━━━━━━━━━━━━━━━━━ 86s 35ms/step - loss: 346.4248 - rmse: 18.6117 - val_loss: 369.9762 - val_rmse: 19.2348 - learning_rate: 0.0010


TEST

In [20]:
test_csv_path =  "/content/drive/MyDrive/UIUC-CS/CS412/Final Project/test.csv"
df_test = pd.read_csv(test_csv_path)
x_test = df_test[['IntersectionId','EntryHeading','ExitHeading']]
x_test = pd.concat([x_test,pd.get_dummies(df_test["City"],dummy_na=False, drop_first=False, dtype=int)],axis=1)
print(x_test.head())
heading_map = {'N':1,'NE':2,'E':3,'SE':4,'S':5, 'SW':6, 'W':7, 'NW': 8}
x_test['EntryHeading'] = x_test['EntryHeading'].replace(heading_map)
x_test['ExitHeading'] = x_test['ExitHeading'].replace(heading_map)
x_test['EntryHeading'] = x_test['EntryHeading'].replace(heading_map)
x_test['ExitHeading'] = x_test['ExitHeading'].replace(heading_map)
print(x_test)

   IntersectionId EntryHeading ExitHeading  Atlanta  Boston  Chicago  \
0               1           NE          NE        1       0        0   
1               1           SW          SE        1       0        0   
2               1           SW          SW        1       0        0   
3               1           NE          NE        1       0        0   
4               1           SW          SW        1       0        0   

   Philadelphia  
0             0  
1             0  
2             0  
3             0  
4             0  


/tmp/ipython-input-4156770442.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x_test['EntryHeading'] = x_test['EntryHeading'].replace(heading_map)


         IntersectionId  EntryHeading  ExitHeading  Atlanta  Boston  Chicago  \
0                     1             2            2        1       0        0   
1                     1             6            4        1       0        0   
2                     1             6            6        1       0        0   
3                     1             2            2        1       0        0   
4                     1             6            6        1       0        0   
...                 ...           ...          ...      ...     ...      ...   
1921352            1990             5            7        0       0        0   
1921353            1990             5            5        0       0        0   
1921354            1990             5            7        0       0        0   
1921355            1990             5            5        0       0        0   
1921356            1990             5            5        0       0        0   

         Philadelphia  
0              

/tmp/ipython-input-4156770442.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x_test['ExitHeading'] = x_test['ExitHeading'].replace(heading_map)


In [21]:
# Hour: 0-23 cyclical -> [sin(2pi*hour/24), cos(2pi*hour/24)]
df_test['hour_sin'] = np.sin(2*np.pi*df_test['Hour']/24)
df_test['hour_cos'] = np.cos(2*np.pi*df_test['Hour']/24)
print(df_test.head())
print(df_test['Month'].unique())

df_test['month_sin'] = np.sin(2*np.pi*df_test['Month']/12)
df_test['month_cos'] = np.cos(2*np.pi*df_test['Month']/12)
print(df_test.head())

   RowId  IntersectionId  Latitude  Longitude             EntryStreetName  \
0      0               1  33.75094 -84.393032  Peachtree Street Southwest   
1      1               1  33.75094 -84.393032  Peachtree Street Southwest   
2      2               1  33.75094 -84.393032  Peachtree Street Southwest   
3      3               1  33.75094 -84.393032  Peachtree Street Southwest   
4      4               1  33.75094 -84.393032  Peachtree Street Southwest   

               ExitStreetName EntryHeading ExitHeading  Hour  Weekend  Month  \
0  Peachtree Street Southwest           NE          NE     0        0      6   
1   Mitchell Street Southwest           SW          SE     0        0      6   
2  Peachtree Street Southwest           SW          SW     0        0      6   
3  Peachtree Street Southwest           NE          NE     1        0      6   
4  Peachtree Street Southwest           SW          SW     1        0      6   

                                                Path    

In [22]:
x_test['hour_sin'] = df_test['hour_sin']
x_test['hour_cos'] = df_test['hour_cos']
x_train['Hour'] = df['Hour']
x_test['month_sin'] = df_test['month_sin']
x_test['month_cos'] = df_test['month_cos']
x_train['Month'] = df['Month']
x_test['Weekend'] = df_test['Weekend']
print(x_test)

         IntersectionId  EntryHeading  ExitHeading  Atlanta  Boston  Chicago  \
0                     1             2            2        1       0        0   
1                     1             6            4        1       0        0   
2                     1             6            6        1       0        0   
3                     1             2            2        1       0        0   
4                     1             6            6        1       0        0   
...                 ...           ...          ...      ...     ...      ...   
1921352            1990             5            7        0       0        0   
1921353            1990             5            5        0       0        0   
1921354            1990             5            7        0       0        0   
1921355            1990             5            5        0       0        0   
1921356            1990             5            5        0       0        0   

         Philadelphia  hour_sin  hour_c

In [18]:
# Make sure x_test has the same columns and order as training
x_test = x_test.reindex(columns=feature_cols, fill_value=0)
print("Aligned x_test shape:", x_test.shape)

# Turn flat x_test into LSTM sequences for test
def make_test_sequences(X_split, seq_len, feature_cols):
    """
    Build one LSTM sequence per test row.
    If there isn't enough history, pad with the first row in that group.
    """
    tmp = X_split.copy()

    city_cols = [c for c in ['Atlanta','Boston','Chicago','Philadelphia'] if c in tmp.columns]
    if len(city_cols) == 0:
        raise ValueError("City one-hot columns not found in x_test.")

    tmp['CityKey'] = tmp[city_cols].idxmax(axis=1)

    for need in ['hour_sin', 'hour_cos', 'month_sin', 'month_cos']:
        if need not in tmp.columns:
            raise ValueError(f"Missing column '{need}' in x_test.")

    tmp['hour_angle']  = np.mod(np.arctan2(tmp['hour_sin'],  tmp['hour_cos']) + 2.0*np.pi, 2.0*np.pi)
    tmp['month_angle'] = np.mod(np.arctan2(tmp['month_sin'], tmp['month_cos']) + 2.0*np.pi, 2.0*np.pi)

    valid_feat_cols = [c for c in feature_cols if c in tmp.columns]

    tmp_sorted = tmp.sort_values(['CityKey', 'IntersectionId', 'month_angle', 'hour_angle'])

    X_list   = []
    row_list = []

    for (_, _), gdf in tmp_sorted.groupby(['CityKey', 'IntersectionId'], sort=False):
        Xg   = gdf[valid_feat_cols].to_numpy(dtype=np.float32)
        idxs = gdf.index.to_numpy()

        for i in range(len(gdf)):
            if i < seq_len - 1:
                need_pad = seq_len - (i + 1)
                pad_part = np.repeat(Xg[0:1, :], need_pad, axis=0)
                window   = np.concatenate([pad_part, Xg[0:i+1, :]], axis=0)
            else:
                window = Xg[i-seq_len+1 : i+1]

            X_list.append(window)
            row_list.append(idxs[i])

    X_seq   = np.stack(X_list).astype(np.float32)
    row_ids = np.array(row_list)

    print("Test sequences shape:", X_seq.shape)
    return X_seq, row_ids

Xte_seq, row_ids_for_seq = make_test_sequences(x_test, SEQ_LEN, feature_cols)

# predict with the trained models
print("Predicting on test sequences...")
pred_time_test = time_model.predict(Xte_seq, batch_size=BATCH_SIZE, verbose=1)  # (M, 3)
pred_dist_test = dist_model.predict(Xte_seq, batch_size=BATCH_SIZE, verbose=1)  # (M, 3)

pred_all_seq = np.concatenate([pred_time_test, pred_dist_test], axis=1)  # (M, 6)

# Map predictions back to original test rows and build submission
n_test = len(df_test)
full_pred = np.zeros((n_test, 6), dtype=np.float32)
full_pred[row_ids_for_seq] = pred_all_seq

rows_out = []
for i in range(n_test):
    for k in range(6):
        target_id  = f"{i}_{k}"
        target_val = full_pred[i, k]
        rows_out.append((target_id, target_val))

sub_df = pd.DataFrame(rows_out, columns=["TargetId", "Target"])
sub_path = "/content/drive/MyDrive/UIUC-CS/CS412/Final Project/submission_lstm.csv"
sub_df.to_csv(sub_path, index=False)

print("Submission saved to:", sub_path)
print(sub_df.head(12))


Raw test shape: (1921357, 13)
   RowId  IntersectionId  Latitude  Longitude             EntryStreetName  \
0      0               1  33.75094 -84.393032  Peachtree Street Southwest   
1      1               1  33.75094 -84.393032  Peachtree Street Southwest   
2      2               1  33.75094 -84.393032  Peachtree Street Southwest   
3      3               1  33.75094 -84.393032  Peachtree Street Southwest   
4      4               1  33.75094 -84.393032  Peachtree Street Southwest   

               ExitStreetName EntryHeading ExitHeading  Hour  Weekend  Month  \
0  Peachtree Street Southwest           NE          NE     0        0      6   
1   Mitchell Street Southwest           SW          SE     0        0      6   
2  Peachtree Street Southwest           SW          SW     0        0      6   
3  Peachtree Street Southwest           NE          NE     1        0      6   
4  Peachtree Street Southwest           SW          SW     1        0      6   

                          

/tmp/ipython-input-82530831.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x_test["EntryHeading"]   = df_test["EntryHeading"].replace(heading_map)
/tmp/ipython-input-82530831.py:20: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x_test["ExitHeading"]    = df_test["ExitHeading"].replace(heading_map)


x_test after feature engineering: (1921357, 14)
   IntersectionId  EntryHeading  ExitHeading  Atlanta  Boston  Chicago  \
0               1             2            2        1       0        0   
1               1             6            4        1       0        0   
2               1             6            6        1       0        0   
3               1             2            2        1       0        0   
4               1             6            6        1       0        0   

   Philadelphia  hour_sin  hour_cos  month_sin  month_cos  Weekend  Month  \
0             0  0.000000  1.000000        0.5  -0.866025        0      6   
1             0  0.000000  1.000000        0.5  -0.866025        0      6   
2             0  0.000000  1.000000        0.5  -0.866025        0      6   
3             0  0.258819  0.965926        0.5  -0.866025        0      6   
4             0  0.258819  0.965926        0.5  -0.866025        0      6   

   Hour  
0     0  
1     0  
2     0  
3   